# Extract features from the news

In [1]:
import TWB
import pandas as pd
import glob

In [2]:
# Make sure that the tags are also wikipedia articles!!!

tags = [ 'humanitarian crisis',
         'natural disaster',
         'environmental crisis',
         'disability',
         'gender',
         'genital mutilation',
         'racism',
         'genocide',
         'civil war',
         'terrorism',
         'infectious disease',
         'political revolution',
         'political prisoner',
         'amnesty',
         'corruption',
         'health',
         'gender inequality',
         'rape',
         'ebola',
         'aids',
         'hiv',
         'technology',
         'artificial intelligence',
         'military',
         'war',
         'climate change',
         'starvation',
         'food shortage',
         'dehydration',
         'water shortage',
         'attack',
         'aggression',
         'logistics',
         'nutrition',
         'protection',
         'shelter',
         'drinking water',
         'sanitation',
         'hygiene',
         'refugee camp',
         'education',
         'emergency communication system',
         'food security',
         'human rights',
         'children',
         'pregnancy',
         'old age',
         'justice',
         'law',
         'maintenance (technical)',
         'homelessness',
         'art',
         'culture',
         'indigenous peoples'
       ]

api_key = '34e73062364d41b180a6f6a9625bff79'

N = TWB.News(tags, api_key)

In [3]:
N.articles # List of articles
N.topics   # Dict of topic -> articles
N.topic_countries # Dict of topic -> List[(ISO3,freq)]

{'humanitarian crisis': [('MEX', 15),
  ('YEM', 13),
  ('SAU', 7),
  ('NOR', 3),
  ('ISR', 3),
  ('AGO', 2),
  ('CMR', 2),
  ('COL', 2),
  ('AUS', 2),
  ('LBY', 2),
  ('NER', 2),
  ('JOR', 2),
  ('CW', 1),
  ('CUB', 1),
  ('LKA', 1),
  ('MLI', 1),
  ('SOM', 1),
  ('ARE', 1),
  ('CHL', 1),
  ('QAT', 1),
  ('RWA', 1),
  ('TCD', 1),
  ('BFA', 1),
  ('JPN', 1),
  ('URY', 1),
  ('NGA', 1),
  ('PER', 1),
  ('BGD', 1),
  ('MMR', 1)],
 'natural disaster': [('PRI', 5),
  ('IND', 2),
  ('GEO', 1),
  ('PAN', 1),
  ('PER', 1),
  ('CHL', 1),
  ('CHE', 1)],
 'environmental crisis': [('DEU', 3),
  ('CAN', 3),
  ('CHN', 2),
  ('FRA', 1),
  ('ITA', 1),
  ('IND', 1),
  ('CUB', 1),
  ('MYS', 1),
  ('BRA', 1),
  ('GBR', 1),
  ('MEX', 1),
  ('JPN', 1),
  ('IRL', 1)],
 'disability': [('IND', 11),
  ('OMN', 3),
  ('ZAF', 1),
  ('IRQ', 1),
  ('MLI', 1),
  ('AFG', 1)],
 'gender': [('OMN', 5),
  ('CAN', 2),
  ('DEU', 2),
  ('JPN', 2),
  ('FRA', 2),
  ('NZL', 1),
  ('IND', 1),
  ('AUS', 1),
  ('DNK', 1),
  ('FIN

# Wikipedia

In [4]:
W = TWB.Wikipedia(tags)

# Dictionary construction

In [5]:
D = TWB.Dictionary([ a.text() for a in W.articles ])

## Distance to news

In [4]:
import glob

data_location = '/Users/thiesgehrmann/Downloads/TWB/documents/hackathon-for-good-2019_TWB-challenge_files/*.sdlxliff'
xliff_files = glob.glob(data_location)
xliff = [ TWB.XLIFF(f) for f in xliff_files[:100]]

In [7]:


def distance_to_articles(xliffs, news, dictionary, dist_metric=None):
    """
    
    """
    
    doc_text  = [ ' '.join(x.source) for x in xliffs ]
    news_text = [ a.text() for a in news.articles ]
    
    return cdist(dictionary.annotate(doc_text), dictionary.annotate(news_text))
#edef
    
def document_topic(xliffs, news, dictionary, k=1, dist_metric=None, dist_to_topic=True):
    """
    if k=1, then 
    """
    dists = distance_to_articles(xliffs, news, dictionary, dist_metric)
    
    articles = news.articles
    
    xliff_topics = []
    
    for xliff_i, d_i in enumerate(dists):
        S = sorted(enumerate(d_i), key=lambda x:x[1])
        if dist_to_topic:
            S = [ (articles[i].topic, d) for (i,d) in S ]
            xliff_topics.append({ t: min(v) for (t,v) in group(S, key=lambda x: x[0], value=lambda x:x[1]).items()})
        else:
            article_indexes, distances = list(zip(*S))
            xliff_topics.append(TWB.common.freq([articles[i].topic for i in article_indexes[:k] ]))
        #fi
    #efor
    
    return xliff_topics
#edef

NT = document_topic(xliff, W, D)

NameError: name 'group' is not defined

In [ ]:
NT

In [37]:
import TWB

from scipy.spatial.distance import cdist

class API(object):
    def __init__(self, news_api_key, tags=tags):
        self.lsc = TWB.LSC()
        self.W = TWB.Wikipedia(tags)
        
        T = [ a.text() for a in self.W.articles ]
        
        self.N  = TWB.News(tags, news_api_key)
        self.TC = self.N.topic_countries
        
        self.D = TWB.Dictionary(T)
        self.A = self.D.annotate(T)
    #edef
    
    def get(self, xliffs, max_topics=10):
        """
        parameters:
        -----------
        xliffs: List[TWB.XLIFF] | XLIFF
        
        max_topics: Integer
            Maximum number of topics to report (sorted by prevalence)
        
        returns:
        dict
        """
        
        if isinstance(xliffs, TWB.XLIFF):
            xliffs = [ xliffs ]
        #fi
        
        XA = [ { 'target_lsc' : self.lsc.detect(i.target_lang) } for i in xliffs ]
        print(len(XA))
        
        topic_languages = self.get_topics_languages(xliffs, max_topics)
        return topic_languages
        for i,v in enumerate(topic_languages):
            print(i)
            XA[i]['topics'] = v
        #efor
        
        return XA
    #edef
    
    def get_topics_languages(self, xliffs, max_topics):
        """
        parameters:
        -----------
        xliffs: List[TWB.XLIFF]
        
        max_topics: Integer
            Maximum number of topics to report (sorted by prevalence)
        
        returns:
        dict
        """

        
        doc_text  = [ ' '.join(x.source) for x in xliffs ]

        dists = cdist(self.D.annotate(doc_text), self.A)
        
        articles = self.W.articles

        xliff_topics = []

        for xliff_i, d_i in enumerate(dists):
            S = sorted(enumerate(d_i), key=lambda x:x[1])
            S = [ (articles[i].topic, d) for (i,d) in S ]
            T = { t: min(v) for (t,v) in TWB.common.group(S, key=lambda x: x[0], value=lambda x:x[1]).items()}
            T = dict(list(sorted(T.items(), key=lambda x:x[1]))[:max_topics])
            xliff_topics.append(T)
        #efor

        xliff_annot = []
        for xliff_i, xt, in enumerate(xliff_topics):
            A = { t : { 'distance' : d,
                        'relevant_country_languages' : {
                            country : { 'frequency' : count, 'language_relevant' : False }
                            for (country, count) in self.TC[t]
                        }
                      } for (t,d) in xt.items() }
                     
            target_lang = self.lsc.detect(xliffs[xliff_i].target_lang)
            for t in A:
                for country in A[t]['relevant_country_languages']:
                    A[t]['relevant_country_languages'][country]['language_relevant'] = target_lang.language.iso3 in self.lsc.country_languages(country)
                #efor
            #efor
            
        #efor
        return xliff_annot
    #edef
#eclass
                    
        
        
        

In [38]:
api = API(api_key)

In [39]:
api.get(xliff[2])

1


{'aids': {'distance': 7.615773105863909,
  'relevant_country_languages': {'CAN': {'frequency': 3,
    'language_relevant': False},
   'PAK': {'frequency': 2, 'language_relevant': False},
   'OMN': {'frequency': 2, 'language_relevant': False},
   'FRA': {'frequency': 1, 'language_relevant': False},
   'KEN': {'frequency': 1, 'language_relevant': False},
   'AUT': {'frequency': 1, 'language_relevant': False},
   'PAN': {'frequency': 1, 'language_relevant': True},
   'CHN': {'frequency': 1, 'language_relevant': False},
   'JPN': {'frequency': 1, 'language_relevant': False},
   'MLI': {'frequency': 1, 'language_relevant': False},
   'AUS': {'frequency': 1, 'language_relevant': False},
   'SLV': {'frequency': 1, 'language_relevant': True}}},
 'hiv': {'distance': 7.615773105863909,
  'relevant_country_languages': {'PAK': {'frequency': 11,
    'language_relevant': False},
   'CHN': {'frequency': 4, 'language_relevant': False},
   'OMN': {'frequency': 4, 'language_relevant': False},
   'AUS': 

In [24]:
api.lsc.detect(xliff[2].target_lang).language.iso3
api.lsc.country_languages('PAN')

['SPA']

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.io import output_notebook

output_notebook()

In [ ]:
fruits = list(word_freq.keys())
counts = list(word_freq.values())

# sorting the bars means sorting the range factors
sorted_fruits = sorted(fruits, key=lambda x: counts[fruits.index(x)])

p = figure(x_range=sorted_fruits, plot_height=500, plot_width=1400, title="Word frequencies",
           toolbar_location="below")

p.vbar(x=fruits, top=counts, width=0.9)
p.xaxis.major_label_orientation = "vertical"
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "25pt"
p.xaxis.major_label_text_font_size = "10pt"
p.y_range.start = 0

show(p)

## Google trends

In [1]:
import pandas as pd
from pytrends.request import TrendReq
import os  
import numpy as np

In [19]:
#functions
def get_interest_and_region(kw_list):
    if not (os.path.isfile('Data/'+str(kw_list[0])+'.pkl') and os.path.isfile('Data/'+str(kw_list[0])+'_regions'+'.pkl')):
        print('Term not downloaded yet, downloading now..')
        pytrend = TrendReq(hl='en-US', tz=360)
        #pytrend = TrendReq(hl='en-US', tz=360, timeout=(10,25), proxies=['https://188.217.7.217',], retries=5, backoff_factor=1)

        search_df = pytrend.get_historical_interest(kw_list, year_start=2014, year_end=2018).reset_index()

        search_df.to_pickle('Data/'+str(kw_list[0])+'.pkl')
        region_df = pytrend.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
        region_df.to_pickle('Data/'+str(kw_list[0])+'_regions'+'.pkl')
        print('term with regional data saved')
        return(search, region_df)
    else:
        print('term has been found..')
        pytrend = TrendReq(hl='en-US', tz=360)   
        term_df = pd.read_pickle('Data/'+str(kw_list[0])+'.pkl')
        region_df = pd.read_pickle('Data/'+str(kw_list[0])+'_regions'+'.pkl')
        print(term_df.head())
        return(term_df, region_df)
        
        
def plot_interest_in_time(df2, kw_list):

    from bokeh.io import show, output_file
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from datetime import datetime as dt
    from bokeh.models import DatetimeTickFormatter

    output_notebook()
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
    TOOLTIPS = [
        ('month', '@x{datetime}'),
        ('Total searches', '@y'),
    ]

    p = figure(title="Monthly-wise total search of term - "+str(kw_list[0]), y_axis_type="linear", plot_height = 400,
               tools = TOOLS, tooltips = TOOLTIPS, plot_width = 800)


    p.xaxis.axis_label = 'Month'
    p.yaxis.axis_label = 'Total search'
    #p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

    p.line(df2.index, df2[str(kw_list[0])],line_color="purple", line_width = 3)

    p.xaxis.formatter=DatetimeTickFormatter(

            days=["%d %B %Y"],
            months=["%d %B %Y"],
            years=["%d %B %Y"],
        )
    # output_file("line_chart.html", title="Line Chart")
    show(p)
#edef

term has been found..
                 date  crisis  isPartial
0 2015-01-07 00:00:00      74      False
1 2015-01-07 01:00:00      73      False
2 2015-01-07 02:00:00      75      False
3 2015-01-07 03:00:00      71      False
4 2015-01-07 04:00:00      68      False


In [21]:
## Downsampling to monthly
df2 = time_df.resample('M', on='date').mean()
print(df2.index)

DatetimeIndex(['2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30',
               '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31',
               '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31',
               '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30',
               '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31',
               '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31',
               '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30',
               '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31',
               '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31',
               '2018-01-31', '2018-02-28'],
              dtype='datetime64[ns]', name='date', freq='M')


In [22]:
plot_interest_in_time(df2,kw_list)

Loading BokehJS ...

In [18]:
# #add year and month
# term_csv['Year'] = term_csv.date.apply(lambda x: x.year)
# term_csv['Month'] = term_csv.date.apply(lambda x: x.month)
# term_csv.head()

In [ ]:
# temp_df = term_csv.groupby(['Year']).mean().reset_index()

# temp_df